# 项目：评估和处理英国电商公司销售数据

## 分析目标

此数据分析的目的是，根据市场销售数据，挖掘畅销产品，以便制定更有效的市场策略来提升营收。  
本实战项目的目的在于练习评估数据干净和整洁度，并基于评估结果，对数据进行清洗，从而得到可供下一步分析的数据。

## 简介

原始数据集记录了一家英国在线零售公司在2010年12月1日至2011年12月9日期间的所有交易情况，涵盖了该公司在全球不同国家和地区的业务数据。该公司主要销售覆盖各个场景的礼品，包括但不限于生日礼品、结婚纪念品、圣诞礼品等等。该公司的客户群体主要包括批发商和个人消费者，其中批发商占据了相当大的比例。  



数据每列的含义如下：

- `InvoiceNo`：发票号码，6位数，作为交易的唯一标识符，如果这个代码以'c'开头，表示这笔交易被取消。
- `StockCode`：产品代码，5位数，作为产品的唯一标识符。
- `Description`：产品名称。
- `Quantity`：产品在交易中的数量。
- `InvoiceData`：发票日期和时间，交易发生的日期和时间。
- `UnitPrice`：单价，价格单位为英镑（£）
- `CustomID`：客户编号，5位数，作为客户的唯一标识符。
- `Country`：国家，客户所居住的国家的名称。

## 读取数据

In [1]:
import pandas as pd

In [2]:
original_data = pd.read_csv("./data.csv", encoding = 'latin1')

In [3]:
original_data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


## 评估数据

在这一部分，我将对在上一部分建立的`original_data`这个DataFrame所包含的数据进行评估。  

评估主要从两个方面进行：结构和内容，即整齐度和干净度。数据的结构性问题指不符合“每列是一个变量，每行是一个观察值，每个单元格是一个值”这三个标准，数据的内容性问题包括存在丢失数据、重复数据、无效数据等。


## 评估数据整齐度

In [4]:
original_data.sample(10)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
364851,568675,20719,WOODLAND CHARLOTTE BAG,30,9/28/2011 13:32,0.72,14298.0,United Kingdom
421297,572933,21733,RED HANGING HEART T-LIGHT HOLDER,12,10/27/2011 10:15,2.95,13458.0,United Kingdom
245638,558621,23182,TOILET SIGN OCCUPIED OR VACANT,24,6/30/2011 16:43,0.83,13745.0,United Kingdom
485772,577646,20652,BLUE POLKADOT LUGGAGE TAG,12,11/21/2011 10:39,1.25,13189.0,United Kingdom
336147,566306,23090,VINTAGE GLASS T-LIGHT HOLDER,24,9/12/2011 9:36,0.83,13265.0,United Kingdom
440059,574502,23241,TREASURE TIN GYMKHANA DESIGN,1,11/4/2011 13:16,2.08,17274.0,United Kingdom
522491,580399,23495,SET OF 3 PANTRY WOODEN SPOONS,6,12/4/2011 11:45,1.25,13471.0,United Kingdom
325384,565439,21754,HOME BUILDING BLOCK WORD,3,9/4/2011 13:56,6.25,14852.0,United Kingdom
393926,570861,23191,BUNDLE OF 3 RETRO NOTE BOOKS,12,10/12/2011 15:27,1.65,14775.0,United Kingdom
148469,549240,23154,SET OF 4 JAM JAR MAGNETS,1,4/7/2011 11:37,2.08,15910.0,United Kingdom


从数据的抽样来看，数据满足“每列是一个变量，每行是一个观察值，每个单元格是一个值”这三个标准，具体来看每行是关于某赏评的交易，每列是交易相关的各个变量，因此该数据不存在结构性问题。

## 评估数据干净度

In [5]:
original_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


可以看出一共有`541909`行数据，`Description`和`CustomerID`两个属性存在缺失值。  

此外，`InvoiceDate`数据类型应该为日期，`CustomerID`数据类型应该为字符串。应当进行数据类型转换。

### 评估缺失数据

In [6]:
original_data[original_data["Description"].isnull()]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
622,536414,22139,NaN,56,12/1/2010 11:52,0.0,NaN,United Kingdom
1970,536545,21134,NaN,1,12/1/2010 14:32,0.0,NaN,United Kingdom
1971,536546,22145,NaN,1,12/1/2010 14:33,0.0,NaN,United Kingdom
1972,536547,37509,NaN,1,12/1/2010 14:33,0.0,NaN,United Kingdom
1987,536549,85226A,NaN,1,12/1/2010 14:34,0.0,NaN,United Kingdom
...,...,...,...,...,...,...,...,...
535322,581199,84581,NaN,-2,12/7/2011 18:26,0.0,NaN,United Kingdom
535326,581203,23406,NaN,15,12/7/2011 18:31,0.0,NaN,United Kingdom
535332,581209,21620,NaN,6,12/7/2011 18:35,0.0,NaN,United Kingdom
536981,581234,72817,NaN,27,12/8/2011 10:33,0.0,NaN,United Kingdom


可以看出有`1454`行数据的`Description`属性为缺失值，同时这些数据的`UnitPrice`属性都为0.0，为了验证猜想，进一步分析是否存在`UnPrice`不为0.0的数据。

In [7]:
original_data[(original_data["Description"].isnull()) & (original_data["UnitPrice"] != 0)]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country


筛选出来结果为0条，说明缺失`Description`的数据，同时也不具备有效的`UnitPrice`。  

`Description`表示产品名，`UnitPrice`表示单价，两者为分析的重要变量，如果两者同时缺失，可以视为不具备有效含义，可以在后续的分析过程中删除。

`CustomerID`变量同样存在缺失值，因此也根据条件提取观察缺失值。

In [8]:
original_data[original_data["CustomerID"].isnull()]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
622,536414,22139,NaN,56,12/1/2010 11:52,0.00,NaN,United Kingdom
1443,536544,21773,DECORATIVE ROSE BATHROOM BOTTLE,1,12/1/2010 14:32,2.51,NaN,United Kingdom
1444,536544,21774,DECORATIVE CATS BATHROOM BOTTLE,2,12/1/2010 14:32,2.51,NaN,United Kingdom
1445,536544,21786,POLKADOT RAIN HAT,4,12/1/2010 14:32,0.85,NaN,United Kingdom
1446,536544,21787,RAIN PONCHO RETROSPOT,2,12/1/2010 14:32,1.66,NaN,United Kingdom
...,...,...,...,...,...,...,...,...
541536,581498,85099B,JUMBO BAG RED RETROSPOT,5,12/9/2011 10:26,4.13,NaN,United Kingdom
541537,581498,85099C,JUMBO BAG BAROQUE BLACK WHITE,4,12/9/2011 10:26,4.13,NaN,United Kingdom
541538,581498,85150,LADIES & GENTLEMEN METAL SIGN,1,12/9/2011 10:26,4.96,NaN,United Kingdom
541539,581498,85174,S/4 CACTI CANDLES,1,12/9/2011 10:26,10.79,NaN,United Kingdom


可以看出，`CustomeID`并不是本次分析的重要变量，`CustomeID`缺失的数据其他属性也具有有效值，因此`CustomeID`值缺失并不影响分析，因此保留此变量为空的观察值。

## 评估重复数据

根据数据变量的含义来看，虽然`InvoiceNo`、`StockCode`和`CustomerID`都是唯一标识符，但一次交易可能包含多件商品，因此`InvoiceNo`可以存在重复，不同交易可以包含同件商品，因此`StockCode`可以存在重复，顾客可以进行多次交易或下单多个商品，因此`CustomerID`也可以存在重复。  

那么针对此数据集，我们无需评估重复数据。

## 评估不一致数据

根据分析，不一致数据可能存在于`Country`变量中，因此我们要查看是否存在不同名称指代同一个国家的观察值。

In [9]:
original_data["Country"].value_counts()

Country
United Kingdom          495478
Germany                   9495
France                    8557
EIRE                      8196
Spain                     2533
Netherlands               2371
Belgium                   2069
Switzerland               2002
Portugal                  1519
Australia                 1259
Norway                    1086
Italy                      803
Channel Islands            758
Finland                    695
Cyprus                     622
Sweden                     462
Unspecified                446
Austria                    401
Denmark                    389
Japan                      358
Poland                     341
Israel                     297
USA                        291
Hong Kong                  288
Singapore                  229
Iceland                    182
Canada                     151
Greece                     146
Malta                      127
United Arab Emirates        68
European Community          61
RSA                         58


观察后可以发现，并无不一致数据。

## 评估无效或错误数据

In [10]:
original_data.describe()

,Quantity,UnitPrice,CustomerID
count,541909.000000,541909.000000,406829.000000
mean,9.552250,4.611114,15287.690570
std,218.081158,96.759853,1713.600303
min,-80995.000000,-11062.060000,12346.000000
25%,1.000000,1.250000,13953.000000
50%,3.000000,2.080000,15152.000000
75%,10.000000,4.130000,16791.000000
max,80995.000000,38970.000000,18287.000000


观察可以发现，`Quantity`和`UnitPrice`变量存在负数，会对后续数值分析产生影响。  

因此我们首先筛选出`Quantity`为负数的观察值进行分析，进一步分析其具体含义。

In [11]:
original_data[original_data["Quantity"] < 0]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
141,C536379,D,Discount,-1,12/1/2010 9:41,27.50,14527.0,United Kingdom
154,C536383,35004C,SET OF 3 COLOURED FLYING DUCKS,-1,12/1/2010 9:49,4.65,15311.0,United Kingdom
235,C536391,22556,PLASTERS IN TIN CIRCUS PARADE,-12,12/1/2010 10:24,1.65,17548.0,United Kingdom
236,C536391,21984,PACK OF 12 PINK PAISLEY TISSUES,-24,12/1/2010 10:24,0.29,17548.0,United Kingdom
237,C536391,21983,PACK OF 12 BLUE PAISLEY TISSUES,-24,12/1/2010 10:24,0.29,17548.0,United Kingdom
...,...,...,...,...,...,...,...,...
540449,C581490,23144,ZINC T-LIGHT HOLDER STARS SMALL,-11,12/9/2011 9:57,0.83,14397.0,United Kingdom
541541,C581499,M,Manual,-1,12/9/2011 10:28,224.69,15498.0,United Kingdom
541715,C581568,21258,VICTORIAN SEWING BOX LARGE,-5,12/9/2011 11:57,10.95,15311.0,United Kingdom
541716,C581569,84978,HANGING HEART JAR T-LIGHT HOLDER,-1,12/9/2011 11:58,1.25,17315.0,United Kingdom


可以发现当`Quantity`变量小于0的观察值的`InvoiceNo`值开头都有`C`，因此是被取消的交易。  

因此为验证猜想，尝试筛选出是否存在`InvoiceNo`值开头为`C`，但是`Quantity`变量小于0的观察值。

In [12]:
original_data[(original_data["Quantity"] < 0) & (original_data["InvoiceNo"].str[0] != 'C')]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
2406,536589,21777,NaN,-10,12/1/2010 16:50,0.0,NaN,United Kingdom
4347,536764,84952C,NaN,-38,12/2/2010 14:42,0.0,NaN,United Kingdom
7188,536996,22712,NaN,-20,12/3/2010 15:30,0.0,NaN,United Kingdom
7189,536997,22028,NaN,-20,12/3/2010 15:30,0.0,NaN,United Kingdom
7190,536998,85067,NaN,-6,12/3/2010 15:30,0.0,NaN,United Kingdom
...,...,...,...,...,...,...,...,...
535333,581210,23395,check,-26,12/7/2011 18:36,0.0,NaN,United Kingdom
535335,581212,22578,lost,-1050,12/7/2011 18:38,0.0,NaN,United Kingdom
535336,581213,22576,check,-30,12/7/2011 18:38,0.0,NaN,United Kingdom
536908,581226,23090,missing,-338,12/8/2011 9:56,0.0,NaN,United Kingdom


可以发现存在`InvoiceNo`值开头为`C`，但是`Quantity`变量小于0的观察值，但这些观察值的`UnitPrice`变量的值都为0。  

因此为验证猜想，尝试筛选出是否存在`InvoiceNo`值开头为`C`，`Quantity`变量小于0并且`UnitPrice`变量的值不为0的观察值。


In [13]:
original_data[(original_data["Quantity"] < 0) & (original_data["InvoiceNo"].str[0] != 'C') & (original_data["UnitPrice"] != 0.0)]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country


可以发现并不存在`InvoiceNo`值开头为`C`，`Quantity`变量小于0并且`UnitPrice`变量的值不为0的观察值。
根据输出结果，猜想得到验证，当Quantity 变量为负数的时候，观察值满足以下条件之一：
- `InvoiceNo`以`C`开头，表示订单被取消
- `UnitPrice`为0，说明单价为0英镑

这些交易数据均不是有效成交数据，不贡献销售，不在后续分析范围内，因此我们将在数据清理步骤，将`Quantity`为负数的观察值删除。

接下来筛选出`UnitPrice`值为负数的观察值进行分析。

In [14]:
original_data[original_data["UnitPrice"] < 0]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
299983,A563186,B,Adjust bad debt,1,8/12/2011 14:51,-11062.06,NaN,United Kingdom
299984,A563187,B,Adjust bad debt,1,8/12/2011 14:52,-11062.06,NaN,United Kingdom


可以看出`UnitPrice`值为负数的观察值均为`Adjust bad debt`，意思是坏账调整。  

这些数据同样不是有数据，不在后续分析范围内，需要被删除。

## 整理数据  

- 把`InvoiceDate`变量的数据类型转换为为日期时间
- 把`CustomerID`变量的数据类型转换为字符串
- 把`Description`变量缺失的观察值删除
- 把`Quantity`变量值为负数的观察值删除
- 把`UnitPrice`变量值为负数的观察值删除

首先创建一个`cleaned_data`变量，并将`original_data`的副本赋值给它，从而将清洗后的数据与原始数据区分开来。

In [15]:
cleaned_data = original_data.copy()
cleaned_data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


把`InvoiceDate`变量的数据类型转换为为日期时间。  

可以用`Dataframe`的`to_datetime`方法。

In [16]:
cleaned_data["InvoiceDate"] = pd.to_datetime(cleaned_data["InvoiceDate"])
cleaned_data["InvoiceDate"]

0        2010-12-01 08:26:00
1        2010-12-01 08:26:00
2        2010-12-01 08:26:00
3        2010-12-01 08:26:00
4        2010-12-01 08:26:00
                 ...        
541904   2011-12-09 12:50:00
541905   2011-12-09 12:50:00
541906   2011-12-09 12:50:00
541907   2011-12-09 12:50:00
541908   2011-12-09 12:50:00
Name: InvoiceDate, Length: 541909, dtype: datetime64[ns]

通过检查可以得知`InvoiceDate`变量的值已经被改为时间数据类型。

把`CustomerID`变量的数据类型转换为字符串。

In [17]:
cleaned_data["CustomerID"] = cleaned_data["CustomerID"].astype(str)
cleaned_data["CustomerID"]

0         17850.0
1         17850.0
2         17850.0
3         17850.0
4         17850.0
           ...   
541904    12680.0
541905    12680.0
541906    12680.0
541907    12680.0
541908    12680.0
Name: CustomerID, Length: 541909, dtype: object

通过检查可以发现，`CustomerID`的数据类型变为`object`，符合预期。  

把`CustomerID`值中的.0删除。


In [18]:
cleaned_data["CustomerID"] = cleaned_data["CustomerID"].str.slice(0,5)
cleaned_data["CustomerID"]

0         17850
1         17850
2         17850
3         17850
4         17850
          ...  
541904    12680
541905    12680
541906    12680
541907    12680
541908    12680
Name: CustomerID, Length: 541909, dtype: object

把`Description`变量缺失的观察值删除

In [19]:
cleaned_data.dropna(subset=["Description"], inplace=True)

In [20]:
cleaned_data["Description"].isnull().sum()

np.int64(0)

把`Quantity`变量值为负数的观察值删除，可以将`Quantity`变量值不为负的观察值筛选出来重新复制给`cleaned_data`。

In [21]:
cleaned_data = cleaned_data[cleaned_data["Quantity"] >= 0]

In [22]:
len(cleaned_data[cleaned_data["Quantity"] < 0])

0

用同样的方法来删除`UnitPrice`变量值为负数的观察值。

In [23]:
cleaned_data = cleaned_data[cleaned_data["UnitPrice"] >= 0]

In [24]:
len(cleaned_data[cleaned_data["UnitPrice"] <0])

0

## 保存清理后数据

将清理后的数据重新命名保存，可以在同一个文件夹下查看。

In [25]:
cleaned_data.to_csv("data_cleaned.csv")